## ちょっと間をおいてまた試す

In [295]:
import warnings; warnings.simplefilter("ignore")

import os, sys, gc, random
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode

from config import *

In [296]:
df = pd.read_feather("./input/df_feature01.feather")

In [297]:
train = df[df["train_test"] == "train"].copy()

In [298]:
org_features = [col for col in train.iloc[:, 0:14].columns.tolist() if col not in ["train_test", "index", "disease"]]
org_features

['Age',
 'Gender',
 'T_Bil',
 'D_Bil',
 'ALP',
 'ALT_GPT',
 'AST_GOT',
 'TP',
 'Alb',
 'AG_ratio',
 'Glb']

### series_featureのrank

In [299]:
for col in org_features:
    df[f"{col}_rank"] = df[col].rank(pct=True)

In [300]:
df.head(2)

,Age,Gender,T_Bil,D_Bil,ALP,ALT_GPT,AST_GOT,TP,Alb,AG_ratio,...,Gender_rank,T_Bil_rank,D_Bil_rank,ALP_rank,ALT_GPT_rank,AST_GOT_rank,TP_rank,Alb_rank,AG_ratio_rank,Glb_rank
0,59,Male,0.786886,0.150498,220.178691,13.467617,21.729246,6.815731,3.112276,1.006802,...,0.589167,0.268333,0.273333,0.495833,0.240000,0.297500,0.333333,0.210000,0.254167,0.510000
1,69,Male,1.003987,0.195625,221.218413,51.033462,64.752323,6.889608,3.051521,0.751346,...,0.589167,0.742500,0.506667,0.531667,0.923333,0.865833,0.450000,0.139167,0.059167,0.936667


### Target Encoding

In [303]:
# df = pd.read(train).append(pd.read(test)) --
df.head(2)

,Age,Gender,T_Bil,D_Bil,ALP,ALT_GPT,AST_GOT,TP,Alb,AG_ratio,...,D_Bil_rank,ALP_rank,ALT_GPT_rank,AST_GOT_rank,TP_rank,Alb_rank,AG_ratio_rank,Glb_rank,Age_digit2_te,Gender_te
0,59,Male,0.786886,0.150498,220.178691,13.467617,21.729246,6.815731,3.112276,1.006802,...,0.273333,0.495833,0.240000,0.297500,0.333333,0.210000,0.254167,0.510000,0.466165,0.461977
1,69,Male,1.003987,0.195625,221.218413,51.033462,64.752323,6.889608,3.051521,0.751346,...,0.506667,0.531667,0.923333,0.865833,0.450000,0.139167,0.059167,0.936667,0.348684,0.474187


In [302]:
from sklearn.model_selection import StratifiedKFold, KFold

train = df[df["train_test"] == "train"]
test = df[df["train_test"] == "test"]

label_name = "disease"
te_cols = ["Age_digit2", "Gender"]

# trainのte_colを初期化 --
for _te_col in te_cols:
    train[_te_col+f"_te"] = 0
    skf = StratifiedKFold(n_splits=4, random_state=42, shuffle=True)
    split = skf.split(train, train[label_name])
    
    # trainに対して、hold-outでtarget encoding --
    for fold, (trn_index, val_index) in enumerate(split):
    
        for _te_col in te_cols:
            _te = train.loc[trn_index, :].groupby(_te_col).mean()[label_name]
    
            _train_val = train.loc[val_index, _te_col].reset_index(drop=False)
            _train_val = pd.merge(_train_val, pd.DataFrame(_te).reset_index().rename(columns={label_name:_te_col+"_te"}), how="left", on=_te_col)
    
            train.loc[_train_val["index"].values, _te_col+"_te"] = _train_val[_te_col+"_te"].values

df = train.append(test)

In [304]:
df.head(2)

,Age,Gender,T_Bil,D_Bil,ALP,ALT_GPT,AST_GOT,TP,Alb,AG_ratio,...,D_Bil_rank,ALP_rank,ALT_GPT_rank,AST_GOT_rank,TP_rank,Alb_rank,AG_ratio_rank,Glb_rank,Age_digit2_te,Gender_te
0,59,Male,0.786886,0.150498,220.178691,13.467617,21.729246,6.815731,3.112276,1.006802,...,0.273333,0.495833,0.240000,0.297500,0.333333,0.210000,0.254167,0.510000,0.466165,0.461977
1,69,Male,1.003987,0.195625,221.218413,51.033462,64.752323,6.889608,3.051521,0.751346,...,0.506667,0.531667,0.923333,0.865833,0.450000,0.139167,0.059167,0.936667,0.348684,0.474187
